### Conexión Python - PostgresSQL
El primer paso es instalar el adaptador, para ello utilizaremos:

In [1]:
%pip install src

  Using cached src-0.0.7.zip (6.3 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build src
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for src (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [71 lines of output]
      C:\Users\maria\AppData\Local\Temp\pip-build-env-vu8hexxl\overlay\Lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_py
      creating build\lib\src
      copying src\__init__.py -> build\lib\src
      running egg_info
      writing src.egg-info\PKG-INFO
      writing dependency_links to src.egg-info\dependency_links.txt
      writing requirements to src.egg-info\requires.txt
      writing top-level names to src.egg-info\top_level.txt
      reading manifest file 'src.egg-info\SOURCES.txt'
      reading manifest template 'MANIFEST.in'
      adding license file 'LICENSE.rst'
      writing manifest file 'src.egg-info\SOURCES.txt'
      Traceback (most recent call last):
        Fil

In [2]:
import psycopg2
from src import sql

Después debemos crear la conexión con la base de datos de PostgresSQL

In [3]:
conn = psycopg2.connect (host = "localhost", 
                        database = "Sakila",
                        user = "postgres", 
                        password = "9366")

Ahora ya podemos generar consultas. Estas consultas deben tener la siguiente estructura:

```python
sql_query = '''Consulta de SQL'''
cursor = conn.cursor() # Creamos una variable con la que iniciamos el cursor
cursor.execute(sql_query) # Ejecutamos la query
rows = cursor.fetchall() # Creamos una variable con el restultado de la query en forma de lista de tuplas
tabla = pd.DataFrame(rows)# Genereamos un DataFrame a partir de la variable anterior
colnames = [desc[0] for desc in cursor.description] # Recuperamos los nombres de las columnas de la BBDD. cursor.description devuelve una tupla de tuplas con la misma longitud que columnas tenemos en nuestro resultado de la query. Además, el primer término de cada tupla es el nombre de la columna, por ello vamos a iterar por las tuplas y a tomar el término de la posición 0 para poder crear nuestra lista con los nombres de las columnas.
tabla = tabla.set_axis(colnames, axis = 1)# Renombramos las columnas del DF creado para que coincidan con las de la BBDD
display(tabla)
```

In [4]:
import pandas as pd

## 2. Ejercicios
2.1. Encuentra el título de las películas en la tabla film que no sean ni "NC-17" ni "G" en cuanto a su clasificación

In [5]:
sql_query = '''
SELECT "title"                   AS "Título",
        "rating"                 AS "Clasificación"
FROM "film"
WHERE "rating" <> 'NC-17' 

'''
cursor = conn.cursor() 
cursor.execute(sql_query) 
rows = cursor.fetchall() 
tabla = pd.DataFrame(rows)
colnames = [desc[0] for desc in cursor.description] 
tabla = tabla.set_axis(colnames, axis = 1)
display(tabla)

,Título,Clasificación
0,ACADEMY DINOSAUR,PG
1,ACE GOLDFINGER,G
2,AFFAIR PREJUDICE,G
3,AFRICAN EGG,G
4,AGENT TRUMAN,PG
...,...,...
785,WRONG BEHAVIOR,PG-13
786,WYOMING STORM,PG-13
787,YENTL IDAHO,R
788,YOUNG LANGUAGE,G


2.2. Número de películas por categoría estrenadas en 2006

In [6]:
sql_query = '''
SELECT COUNT("title")                   AS "Número de películas",
        "rating"                 AS "Clasificación"
FROM "film"
WHERE "release_year" = 2006
GROUP BY "rating" 

'''
cursor = conn.cursor() 
cursor.execute(sql_query) 
rows = cursor.fetchall() 
tabla = pd.DataFrame(rows)
colnames = [desc[0] for desc in cursor.description] 
tabla = tabla.set_axis(colnames, axis = 1)
display(tabla)

,Número de películas,Clasificación
0,194,PG
1,223,PG-13
2,195,R
3,178,G
4,210,NC-17


2.3. Averigua el número de alquileres registrados por mes.

In [7]:
sql_query = '''
SELECT EXTRACT(MONTH FROM "rental_date")   AS "Mes de alquiler",
       COUNT("rental_id")
FROM "rental"
GROUP BY "Mes de alquiler" 
ORDER BY "Mes de alquiler"

'''
cursor = conn.cursor() 
cursor.execute(sql_query) 
rows = cursor.fetchall() 
tabla = pd.DataFrame(rows)
colnames = [desc[0] for desc in cursor.description] 
tabla = tabla.set_axis(colnames, axis = 1)
display(tabla)

,Mes de alquiler,count
0,2,182
1,5,1156
2,6,2311
3,7,6709
4,8,5686


2.4. Obtener todas las películas y mostrar los actores que han actuado en ellas, incluso si algunas películas no tienen actores asociados.

In [8]:
sql_query = '''
SELECT "f"."title"  AS "Películas",
       CONCAT("a"."first_name", ' ',"a"."last_name")  AS "Actores"
       
FROM "film" AS "f"
LEFT JOIN "film_actor" AS "fa"
ON "f"."film_id" = "fa"."film_id"
INNER JOIN "actor" AS "a"
ON "fa"."actor_id" = "a"."actor_id"
ORDER BY "title"

'''
cursor = conn.cursor() 
cursor.execute(sql_query) 
rows = cursor.fetchall() 
tabla = pd.DataFrame(rows)
colnames = [desc[0] for desc in cursor.description] 
tabla = tabla.set_axis(colnames, axis = 1)
display(tabla)

,Películas,Actores
0,ACADEMY DINOSAUR,ROCK DUKAKIS
1,ACADEMY DINOSAUR,MARY KEITEL
2,ACADEMY DINOSAUR,JOHNNY CAGE
3,ACADEMY DINOSAUR,PENELOPE GUINESS
4,ACADEMY DINOSAUR,SANDRA PECK
...,...,...
5457,ZOOLANDER FICTION,JADA RYDER
5458,ZOOLANDER FICTION,WHOOPI HURT
5459,ZORRO ARK,LISA MONROE
5460,ZORRO ARK,IAN TANDY


2.5. Obtén los 5 clientes españoles que más dinero se hayan gastado con nosotros.


In [9]:
sql_query = '''
SELECT CONCAT("c"."first_name", ' ', "c"."last_name") AS "Nombre_Cliente",
      SUM("p"."amount" ) AS "Cantidad_gastada"
FROM "payment" AS "p"
INNER JOIN "customer" AS "c" 
ON "p"."customer_id" = "c"."customer_id"
INNER JOIN "address" AS "a" 
ON "c"."address_id" = "a"."address_id" 
INNER JOIN "city" AS "ci" 
ON "a"."city_id"= "ci"."city_id" 
INNER JOIN "country" AS "co"
ON "ci"."country_id" = "co"."country_id"
WHERE "co"."country" = 'Spain'
GROUP BY "Nombre_Cliente"
ORDER BY "Cantidad_gastada" DESC
LIMIT 5

'''
cursor = conn.cursor() 
cursor.execute(sql_query) 
rows = cursor.fetchall() 
tabla = pd.DataFrame(rows)
colnames = [desc[0] for desc in cursor.description] 
tabla = tabla.set_axis(colnames, axis = 1)
display(tabla)

,Nombre_Cliente,Cantidad_gastada
0,WESLEY BULL,177.60
1,MAE FLETCHER,158.69
2,JULIE SANCHEZ,107.71
3,CHRIS BROTHERS,84.78
4,NELSON CHRISTENSON,77.80


2.6. Encuentra el nombre y apellido de los actores que no han actuado en ninguna película de la categoría "Music". 


In [10]:
sql_query = '''
SELECT CONCAT("a"."first_name", ' ', "a"."last_name") AS "Nombre_actores"
FROM "actor" AS "a"
WHERE "a"."actor_id" NOT IN (
    SELECT "fa"."actor_id" 
    FROM "film_actor" AS "fa" 
    WHERE "fa"."film_id" IN (
      SELECT "fc"."film_id" 
      FROM "film_category" AS "fc" 
      WHERE "fc"."category_id" IN (
        SELECT "c"."category_id" 
        FROM "category" AS "c"
        WHERE "name"='Music'
      )
    )
)
'''
cursor = conn.cursor() 
cursor.execute(sql_query) 
rows = cursor.fetchall() 
tabla = pd.DataFrame(rows)
colnames = [desc[0] for desc in cursor.description] 
tabla = tabla.set_axis(colnames, axis = 1)
display(tabla)

,Nombre_actores
0,BETTE NICHOLSON
1,GRACE MOSTEL
2,CUBA OLIVIER
3,FRED COSTNER
4,ELVIS MARX
5,CAMERON STREEP
6,RIP CRAWFORD
7,SISSY SOBIESKI
8,MILLA PECK
9,JUDY DEAN


2.7. Encuentra el título de todas las películas que son de la misma categoría que "Animation".

In [11]:
sql_query = '''
SELECT "f"."title" AS "Películas"
FROM "film" AS "f"
WHERE "f"."film_id" IN (
      SELECT "fc"."film_id" 
      FROM "film_category" AS "fc" 
      WHERE "fc"."category_id" IN (
        SELECT "c"."category_id" 
        FROM "category" AS "c"
        WHERE "name"='Animation'
      )
)

'''
cursor = conn.cursor() 
cursor.execute(sql_query) 
rows = cursor.fetchall() 
tabla = pd.DataFrame(rows)
colnames = [desc[0] for desc in cursor.description] 
tabla = tabla.set_axis(colnames, axis = 1)
display(tabla)

,Películas
0,SONS INTERVIEW
1,DOUBLE WRATH
2,ROOM ROMAN
3,BIKINI BORROWERS
4,THIEF PELICAN
...,...
61,PRIDE ALAMO
62,ALTER VICTORY
63,INCH JET
64,JUGGLER HARDLY


2.8. Encuentra los nombres de los actores que han actuado en al menos una película que pertenece a la categoría "Sci-Fi." Ordena los resultados alfabéticamente por apellido.

In [12]:
sql_query = '''
SELECT CONCAT("a"."first_name", ' ', "a"."last_name") AS "Nombre_actores"
FROM "actor" AS "a"
WHERE "a"."actor_id" IN (
    SELECT "fa"."actor_id" 
    FROM "film_actor" AS "fa" 
    WHERE "fa"."film_id" IN (
      SELECT "fc"."film_id" 
      FROM "film_category" AS "fc" 
      WHERE "fc"."category_id" IN (
        SELECT "c"."category_id" 
        FROM "category" AS "c"
        WHERE "name"='Sci-Fi'
    ORDER BY "a"."last_name"
      )
    )
)

'''
cursor = conn.cursor() 
cursor.execute(sql_query) 
rows = cursor.fetchall() 
tabla = pd.DataFrame(rows)
colnames = [desc[0] for desc in cursor.description] 
tabla = tabla.set_axis(colnames, axis = 1)
display(tabla)

,Nombre_actores
0,PENELOPE GUINESS
1,NICK WAHLBERG
2,ED CHASE
3,JENNIFER DAVIS
4,JOHNNY LOLLOBRIGIDA
...,...
162,JAYNE SILVERSTONE
163,BELA WALKEN
164,REESE WEST
165,MARY KEITEL


## Chinook


In [14]:
sql.conexion_Chinook(''' SELECT*
                          FROM "Artist"
                          LIMIT 5
                          ''')

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
